# Final Assignment

- Finish shakespeare (but add more training examples where you're not just predicting the end of a sentence, but also next words) and produce a nice sounding sonnet. We'll read each others' sonnets in class.

- Do a corpus in your own language

- Finish training addition with the tensorflow RNN (one-hot encoded)

- Train a *dense* network to add two numbers

- Start working on your final project: An RNN/CNN to predict LES snow.

- What's the difference between **dense** networks and **recurrent** networks, in your words? Please be able to answer this question!

In [ ]:
import csv
import itertools
import operator
import numpy as np
import nltk
import sys
from datetime import datetime

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
nltk.download('punkt')

In [ ]:
import re
from nltk import tokenize

#alphabets= "([A-Za-z])"
#prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
#suffixes = "(Inc|Ltd|Jr|Sr|Co)"
#starters = "(Mr|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
#acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
#websites = "[.](com|net|org|io|gov|edu|me)"
#digits = "([0-9])"

# vocabulary_size = 8000
vocabulary_size = 3000

unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

def clean_roman_numerals(text):
    pattern = r"\b(?=[MDCLXVIΙ])M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})([IΙ]X|[IΙ]V|V?[IΙ]{0,3})\b\.?"
    return re.sub(pattern, '&', text)

# Read the data and append SENTENCE_START and SENTENCE_END tokens
print( "Reading txt file...")
with open(r'/content/shakespeare-sonnets.txt', 'r', encoding="ISO-8859-1") as f:
    text = f.read()
    
    text = text.replace(",",".")
    text = text.replace(":",".")
    text = text.replace(";",".")
    text = text.replace("?",".")
    text = text.replace("!",".")
    
    text = clean_roman_numerals(text)
    
    #text = re.sub(prefixes,"\\1<prd>",text)
    #text = re.sub(websites,"<prd>\\1",text)
    #text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    #if "..." in text: text = text.replace("...","<prd><prd><prd>")
    #if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    #text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    #text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    #text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    #text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    #text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    #text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    #text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    #if "”" in text: text = text.replace(".”","”.")
    #if "\"" in text: text = text.replace(".\"","\".")
    #if "!" in text: text = text.replace("!\"","\"!")
    #if "?" in text: text = text.replace("?\"","\"?")
    #text = text.replace(".",".<stop>")
    #text = text.replace("?","?<stop>")
    #text = text.replace("!","!<stop>")
    #text = text.replace("<prd>",".")
    #sentences = text.split("<stop>")
    #sentences = sentences[:-1]
    #sentences = [s.strip() for s in sentences]
    
    sentences = tokenize.sent_tokenize(text)
    
    # Append SENTENCE_START and SENTENCE_END
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
    
print(  "Parsed %d sentences." % (len(sentences)))
    
# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print(  "Found %d unique words tokens." % len(word_freq.items()))

# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

print("Using vocabulary size %d." % vocabulary_size)
print("The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1]))

# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

print(  "\nExample sentence: '%s'" % sentences[0])
print(  "\nExample sentence after Pre-processing: '%s'" % tokenized_sentences[0])

In [ ]:
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])

In [ ]:
X_train.shape, y_train.shape

In [ ]:
class RNN:    
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        # Assign instance variables
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        
        # Randomly initialize the network parameters
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))

In [ ]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [ ]:
def forward_propagation(self, x):
    # The total number of time steps
    T = len(x)
    
    # During forward propagation we save all hidden states in s because need them later.
    # We add one additional element for the initial hidden, which we set to 0
    s = np.zeros((T + 1, self.hidden_dim))
    s[-1] = np.zeros(self.hidden_dim)
    
    # The outputs at each time step. Again, we save them for later.
    o = np.zeros((T, self.word_dim))
    
    # For each time step...
    for t in np.arange(T):
        # Note that we are indxing U by x[t]. This is the same as multiplying U with a one-hot vector.
        s[t] = np.tanh(self.U[:,x[t]] + self.W.dot(s[t-1]))
        o[t] = softmax(self.V.dot(s[t]))
        
    return [o, s]

RNN.forward_propagation = forward_propagation

We not only return the calculated outputs, but also the hidden states. We will use them later to calculate the gradients, and by returning them here we avoid duplicate computation. Each `o`  is a vector of probabilities representing the words in our vocabulary, but sometimes, for example when evaluating our model, all we want is the next word with the highest probability. We call this function `predict`:

In [ ]:
def predict(self, x):
    # Perform forward propagation and return index of the highest score
    o, s = self.forward_propagation(x)
    return np.argmax(o, axis=1)

RNN.predict = predict

Let's try and see an example output:

In [ ]:
X_train.shape

In [ ]:
print ("x:\n%s\n%s" % (" ".join([index_to_word[x] for x in X_train[10]]), X_train[10]))

In [ ]:
np.random.seed(17)
model = RNN(vocabulary_size)
o, s = model.forward_propagation(X_train[10])
print (o.shape, o)

For each word in the sentence (16 above), our model made 8000 predictions representing probabilities of the next word. 

That is because we picked a vocabulary size $C = 8000$, so our output layer has $8000$ neurons.

The following gives the indices of the highest probability predictions for each word:

In [ ]:
predictions = model.predict(X_train[10])
print (predictions.shape, predictions)
print ("x:\n%s" % (" ".join([index_to_word[x] for x in predictions])))

Ok, all parameters are as of yet untrained.

## Calculating the Loss

To train our network we need a way to measure the errors it makes. We call this the loss function $L$, and our goal is find the parameters $U,V$ and $W$ that minimize the loss function for our training data. 

A common choice for the loss function is the [cross-entropy loss](https://en.wikipedia.org/wiki/Cross_entropy#Cross-entropy_error_function_and_logistic_regression). 

If we have $N$ training examples (words in our text) and $C$ classes (the size of our vocabulary) then the loss with respect to our predictions $o$ and the true labels $y$ is given by:

$
\begin{aligned}
L(y,o) = - \frac{1}{N} \sum_{n \in N} y_{n} \log o_{n}
\end{aligned}
$

The formula looks a bit complicated, but all it really does is sum over our training examples and add to the loss based on how off our prediction are. The further away $y$ (the correct words) and $o$ (our predictions), the greater the loss will be. We implement the function `calculate_loss`:

In [ ]:
def calculate_total_loss(self, x, y):
    L = 0
    # For each sentence...
    for i in np.arange(len(y)):
        o, s = self.forward_propagation(x[i])
        # We only care about our prediction of the "correct" words
        correct_word_predictions = o[np.arange(len(y[i])), y[i]]
        # Add to the loss based on how off we were
        L += -1 * np.sum(np.log(correct_word_predictions))
    return L

def calculate_loss(self, x, y):
    # Divide the total loss by the number of training examples
    N = np.sum((len(y_i) for y_i in y))
    return self.calculate_total_loss(x,y)/N

RNN.calculate_total_loss = calculate_total_loss
RNN.calculate_loss = calculate_loss

What should the loss be should be for random predictions? 

That will give us a baseline and make sure our implementation is correct. 

We have $C$ words in our vocabulary, so each word should be (on average) predicted with probability $1/C$, which would yield a loss of $L = -\frac{1}{N} N \log\frac{1}{C} = \log C$:

In [ ]:
# Limit to 1000 examples to save time
print ("Expected Loss for random predictions: %f" % np.log(vocabulary_size))
print ("Actual loss: %f" % model.calculate_loss(X_train, y_train))

## Training with Backpropagation Through Time

We iterate over all our training examples and during each iteration we nudge the parameters into a direction that reduces the error. 

These directions are given by the gradients on the loss: $\frac{\partial L}{\partial U}, \frac{\partial L}{\partial V}, \frac{\partial L}{\partial W}$. 

We also need a *learning rate*, which defines how big of a step we want to make in each iteration. 

Because the layer weight parameters are shared by all time steps in the network, the gradient at each output depends not only on the calculations of the current time step, but also the previous time steps!

We take as input a training example $(x,y)$ and return the gradients $\frac{\partial L}{\partial U}, \frac{\partial L}{\partial V}, \frac{\partial L}{\partial W}$.

In [ ]:
def bptt(self, x, y):
    T = len(y)
    
    # Perform forward propagation
    o, s = self.forward_propagation(x)
    
    # We accumulate the gradients in these variables
    dLdU = np.zeros(self.U.shape)
    dLdV = np.zeros(self.V.shape)
    dLdW = np.zeros(self.W.shape)
    delta_o = o
    delta_o[np.arange(len(y)), y] -= 1.
    
    # For each output backwards...
    for t in np.arange(T)[::-1]:
        dLdV += np.outer(delta_o[t], s[t].T)
        
        # Initial delta calculation
        delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
        
        # Backpropagation through time (for at most self.bptt_truncate steps)
        for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
            
            # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
            dLdW += np.outer(delta_t, s[bptt_step-1])              
            dLdU[:,x[bptt_step]] += delta_t
            
            # Update delta for next step
            delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
            
    return [dLdU, dLdV, dLdW]

RNN.bptt = bptt

## Gradient Checking

Whenever you implement backpropagation it is good idea to also implement *gradient checking*, which is a way of verifying that your implementation is correct. The idea behind gradient checking is that derivative of a parameter is equal to the slope at the point, which we can approximate by slightly changing the parameter and then dividing by the change:

$
\begin{aligned}
\frac{\partial L}{\partial \theta} \approx \lim_{h \to 0} \frac{J(\theta + h) - J(\theta -h)}{2h}
\end{aligned}
$

We then compare the gradient we calculated using backpropagation to the gradient we estimated with the method above. If there's no large difference we are good. The approximation needs to calculate the total loss for *every* parameter, so that gradient checking is very expensive (remember, we had more than a million parameters in the example above). So it's a good idea to perform it on a model with a smaller vocabulary.

In [ ]:
def gradient_check(self, x, y, h=0.001, error_threshold=0.01):
    
    # Calculate the gradients using backpropagation. We want to checker if these are correct.
    bptt_gradients = model.bptt(x, y)
    
    # List of all parameters we want to check.
    model_parameters = ['U', 'V', 'W']
    
    # Gradient check for each parameter
    for pidx, pname in enumerate(model_parameters):
        
        # Get the actual parameter value from the mode, e.g. model.W
        parameter = operator.attrgetter(pname)(self)
        print("Performing gradient check for parameter %s with size %d." % (pname, np.prod(parameter.shape)))
               
        # Iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
        it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            ix = it.multi_index
               
            # Save the original value so we can reset it later
            original_value = parameter[ix]
               
            # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
            parameter[ix] = original_value + h
            gradplus = model.calculate_total_loss([x],[y])
            parameter[ix] = original_value - h
            gradminus = model.calculate_total_loss([x],[y])
            estimated_gradient = (gradplus - gradminus)/(2*h)
               
            # Reset parameter to original value
            parameter[ix] = original_value
               
            # The gradient for this parameter calculated using backpropagation
            backprop_gradient = bptt_gradients[pidx][ix]
               
            # calculate The relative error: (|x - y|/(|x| + |y|))
            relative_error = np.abs(backprop_gradient - estimated_gradient) / (
                                np.abs(backprop_gradient) + np.abs(estimated_gradient))
            
               # If the error is to large fail the gradient check
            if relative_error > error_threshold:
                print( "Gradient Check ERROR: parameter=%s ix=%s" % (pname, ix))
                print( "+h Loss: %f" % gradplus)
                print( "-h Loss: %f" % gradminus)
                print( "Estimated_gradient: %f" % estimated_gradient)
                print( "Backpropagation gradient: %f" % backprop_gradient)
                print( "Relative Error: %f" % relative_error)
                return 
            it.iternext()
               
        print( "Gradient check for parameter %s passed." % (pname))

RNN.gradient_check = gradient_check

## Gradient Descent Implementation

A function `sdg_step` calculates the gradients and performs the updates for one batch. 

Then, an outer loop that iterates through the training set and adjusts the learning rate.

In [ ]:
# Performs one step of SGD.
def numpy_sdg_step(self, x, y, learning_rate):
    # Calculate the gradients
    dLdU, dLdV, dLdW = self.bptt(x, y)
    
    # Change parameters according to gradients and learning rate
    self.U -= learning_rate * dLdU
    self.V -= learning_rate * dLdV
    self.W -= learning_rate * dLdW

RNN.sgd_step = numpy_sdg_step

In [ ]:
# Outer SGD Loop
# - model: The RNN model instance
# - X_train: The training data set
# - y_train: The training data labels
# - learning_rate: Initial learning rate for SGD
# - nepoch: Number of times to iterate through the complete dataset
# - evaluate_loss_after: Evaluate the loss after this many epochs

def train_with_sgd(model, X_train, y_train, learning_rate=0.005, nepoch=100, evaluate_loss_after=5):
    # We keep track of the losses so we can plot them later
    losses = []
    num_examples_seen = 0
    
    for epoch in range(nepoch):
        
        # Optionally evaluate the loss
        if (epoch % evaluate_loss_after == 0):
            loss = model.calculate_loss(X_train, y_train)
            losses.append((num_examples_seen, loss))
            time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print ("%s: Loss after num_examples_seen=%d epoch=%d: %f" % (time, num_examples_seen, epoch, loss))
            
            # Adjust the learning rate if loss increases
            if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                learning_rate = learning_rate * 0.5  
                print ("Setting learning rate to %f" % learning_rate)
            sys.stdout.flush()
            
        # For each training example...
        for i in range(len(y_train)):
            
            # One SGD step
            model.sgd_step(X_train[i], y_train[i], learning_rate)
            num_examples_seen += 1

Let's get a sense of how long it would take to train our network:

In [ ]:
np.random.seed(21)
model = RNN(vocabulary_size)
%timeit model.sgd_step(X_train[20], y_train[20], 0.005)

In [ ]:
np.random.seed(10)

# Train on a small subset of the data to see what happens
model = RNN(vocabulary_size)
losses = train_with_sgd(model, X_train, y_train, nepoch=10, evaluate_loss_after=1)



## Generating Text

Now that we have a model, albeit not a very trained one, we can ask it to generate new text for us! Let's implement a helper function to generate new sentences:

In [ ]:
def generate_sentence(model, senten_max_length):
    # We start the sentence with the start token
    new_sentence = [word_to_index[sentence_start_token]]
    
    # Repeat until we get an end token and keep our sentences to less than senten_max_length words for now
    while (not new_sentence[-1] == word_to_index[sentence_end_token]) and len(new_sentence) < senten_max_length:
        next_word_probs = model.forward_propagation(new_sentence)
        sampled_word = word_to_index[unknown_token]
        
        # We don't want to sample unknown words
        while sampled_word == word_to_index[unknown_token]:

            samples = np.random.multinomial(1, next_word_probs[0][-1])
            sampled_word = np.argmax(samples)
            
        new_sentence.append(sampled_word)    
        sentence_str = [index_to_word[x] for x in new_sentence[1:-1]]
    return sentence_str

In [ ]:
num_sentences = 10
senten_min_length = 7
senten_max_length = 20

for i in range(num_sentences):
    sent = []
    # We want long sentences, not sentences with one or two words
    while len(sent) < senten_min_length:
        sent = generate_sentence(model, senten_max_length)
    print (" ".join(sent))

# Question 2 RNN with tenserflow

In [ ]:
from __future__ import print_function
#from keras.models import Sequential
#from keras import layers
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import numpy as np
from six.moves import range

# A one-hot encoding python class

In [ ]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.

        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.

        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

# Generating training data



In [ ]:
''.join(np.random.choice(list('0123456789')))

In [ ]:
int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, 4))))

In [ ]:
np.random.choice(list('0123456789'))

In [ ]:
# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = False

# Maximum length of input is 'int + int' (e.g., '34+78  '). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    
    # store
    questions.append(query)
    expected.append(ans)
    print(query, ans)
    
print('Total addition questions:', len(questions))

Let's list our **input** sequence:

In [ ]:
questions

Let's list our **output** sequence:

In [ ]:
expected

# Training and test data

Let's divide all our observations into a *training* set and a *test* set. We will now *encode* our numbers into **one-hot vectors**: 

The first dimension is the numbers of rows (observations).

The second dimension is the number of characters in each sequence.

The third dimension is the size of our alphabet of symbols. Here, it's `len('0123456789+ ')`. In our statistical sentence translation notebook, it's the size of the word dictionary (word-to-index mapping).

Let's allocate space for the input to the encoder `x`, and the input to the decoder, `y`. They're both 3D **tensors**: The first dimension is the row, and the other two dimensions are the thought-vector representation of the question or the answer.

Here's the first question in math format, and in thought-vector format:

In [ ]:
ctable.encode(questions[0], MAXLEN)

And here's the answer, in math format, and in thought-vector format:

In [ ]:
ctable.encode(expected[0], DIGITS + 1)

Let's create the thought-vectors for our entire training dataset:

In [ ]:
print('Vectorization into thought:')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)
print()

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)
print()

print('Example:')
print('The first row of input data is encoded internally as:')
print(x_train[0])
print()
print('The first row of output data is encoded internally as:')
print(y_train[0])
print()
print('These internal representations represent these signals:')
print(ctable.decode(x_train[0]))
print(ctable.decode(y_train[0]))

## RNN

In [ ]:
# Try replacing with GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

print('Build model...')
model = Sequential()

# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))

# As the decoder RNN's input, repeatedly provide with the last output of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))

# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
# We require DIGITS + 1 output vectors for our result. We will use the same fully 
# connected layer (Dense) to output each vector. To use the same layer DIGITS + 1
# times, we wrap it in a TimeDistributed() wrapper layer
model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

We had to connect the encoder to the decoder and they *did not originally fit*!

With
```(python)
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
```

..the encoder takes in a **3D tensor** of 50,000 rows, MAXLEN timesteps, `len(char)` features, and...

..for each observation, the encoder will produce a **2D matrix** of 128 rows and `len(chars)` columns (after MAXLEN timesteps), while the decoder needs a **3D tensor**, not just 128 rows and `len(chars)` features, which it will then run over `DIGITS + 1` timesteps. That's a problem!

Keras' [RepeatVector](https://keras.io/layers/core/#repeatvector) layer is used like an adapter to fit the encoder and decoder. We configure `RepeatVector` to repeat the input `DIGITS + 1` times. This creates a 3D output comprised of `DIGITS + 1` copies of 128 x `len(char)` features, that we decode `DIGITS + 1` times using the same fully connected layer for each of the `DIGITS + 1` desired output vectors.

If you comment out the `RepeatVector` line, you will get:
```(python)
ValueError: Input 0 is incompatible with layer lstm_xx: expected ndim=3, found ndim=2
```
Try it out!

Finally, the fully connected output layer will use a softmax activation function to output values in the range \[0,1\], so some closer to `False`, and others closer to` True`. We will approximate by **rounding off** to 0 or 1 for our final prediction. 

>**Note**: Why not leverage one output for *each* time step for the input sequence, rather than one output for the *final* timestep of the input sequence (which is what we do here)? The alternate option would give us intermediate timestep information, which we don't leveerage here. This is discussed [here](https://github.com/fchollet/keras/issues/395).

# Training

Let's train our RNN for 35 epochs.

In [ ]:
# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 2):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    
    # Select 10 samples from the validation set at random so we can visualize
    # errors with green and red boxes
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        #preds = model.predict_classes(rowx, verbose=0)
        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        print("debugging:")
        print(type(preds[0]))
        print("now decoding...:")
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

In [ ]:
# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 35):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    
    # Select 10 samples from the validation set at random so we can visualize
    # errors with green and red boxes
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        print("debugging:")
        print(type(preds[0]))
        print("now decoding...:")
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

The loss is pretty low after only 30 epochs (and the samples are all correct), so I stopped the training!

Let's evaluate the accuracy of our model:

In [ ]:
# evaluate the keras model
_, accuracy = model.evaluate(x_val, y_val)
print('Accuracy: %.2f' % (accuracy*100))

99%.. Not bad.. We now know how to add numbers!

Let's add two numbers and test our network. We need to make sure we encode our numbers as vectors of the same type and shape used for training!

Let's pick two numbers at random, sum them up, and pad them to 7 characters long:

In [ ]:
x = np.random.randint(0, 100)
y = np.random.randint(0, 100)
z = x + y
print(x,y,z)
print()

x_plus_y_buffered = str(x) + '+' + str(y) + (7 - len(str(x) + '+' + str(y))) * ' '
x_plus_y_buffered

Let's do the same thing with the result:

In [ ]:
z_buffered = str(z) + (4 - len(str(z))) * ' '
z_buffered

Now let's encode these numbers in the internal representation of the neural net.

>**Note**: Every kind of sensory inpur, whether it's from your eyes, your ears, your nose,... get encoded in *internal* brain representations. So, we're doing exactly the same thing here. Except that the internal representations of sensory inputs in our brains happen at a darwinian scale. That goes to show that finding the *right* encoding may make a big difference in the training and have an effect on overall accuracy.

In [ ]:
x_plus_y_encoded = ctable.encode(x_plus_y_buffered, 7)
x_plus_y_encoded

In [ ]:
ctable.decode(x_plus_y_encoded)

In [ ]:
z_encoded = ctable.encode(z_buffered, 4)
z_encoded

In [ ]:
ctable.decode(z_encoded)

We need to convert to booleans, since that is how we trained our RNN:

In [ ]:
x_plus_y = str(x) + '+' + str(y)
x_plus_y_tf = np.zeros((len(x_plus_y_buffered), len(chars)), dtype=np.bool)
x_plus_y_tf

In [ ]:
list(enumerate(x_plus_y_tf))

In [ ]:
ctable.encode('2', 1)

In [ ]:
for i, sentence in enumerate(x_plus_y_tf):
    x_plus_y_tf[i] = ctable.encode(x_plus_y_buffered[i], 1)
x_plus_y_tf

In [ ]:
z_tf = np.zeros((len(z_buffered), len(chars)), dtype=np.bool)
z_tf

In [ ]:
for i, sentence in enumerate(z_tf):
    z_tf[i] = ctable.encode(z_buffered[i], 1)
z_tf

We're now ready for a **forward step** through our RNN. `Keras`' `predict_classes()` API takes in an *array* of inputs, so we arbitrarily double up our input: 

In [ ]:
x_plus_y_tf2 = np.array((x_plus_y_tf, x_plus_y_tf), dtype=np.bool)
x_plus_y_tf2

We're now ready to call `predict_classes()`:

In [ ]:
preds = np.argmax(model.predict(x_plus_y_tf2), axis=-1)
preds

Let's decode that representation:

In [ ]:
ctable.decode(preds[0], calc_argmax=False)

In [ ]:
# evaluate the keras model
_, accuracy = model.evaluate(x_val, y_val)
print('Accuracy: %.2f' % (accuracy*100))



x = np.random.randint(0, 100)
y = np.random.randint(0, 100)
z = x + y
print(x,y,z)
print()

x_plus_y_buffered = str(x) + '+' + str(y) + (7 - len(str(x) + '+' + str(y))) * ' '
x_plus_y_buffered



z_buffered = str(z) + (4 - len(str(z))) * ' '
z_buffered

x_plus_y_encoded = ctable.encode(x_plus_y_buffered, 7)
x_plus_y_encoded

ctable.decode(x_plus_y_encoded)

z_encoded = ctable.encode(z_buffered, 4)
z_encoded

ctable.decode(z_encoded)



x_plus_y = str(x) + '+' + str(y)
x_plus_y_tf = np.zeros((len(x_plus_y_buffered), len(chars)), dtype=np.bool)
x_plus_y_tf

list(enumerate(x_plus_y_tf))

ctable.encode('2', 1)

for i, sentence in enumerate(x_plus_y_tf):
    x_plus_y_tf[i] = ctable.encode(x_plus_y_buffered[i], 1)
x_plus_y_tf

z_tf = np.zeros((len(z_buffered), len(chars)), dtype=np.bool)
z_tf

for i, sentence in enumerate(z_tf):
    z_tf[i] = ctable.encode(z_buffered[i], 1)
z_tf


x_plus_y_tf2 = np.array((x_plus_y_tf, x_plus_y_tf), dtype=np.bool)
x_plus_y_tf2



preds = np.argmax(model.predict(x_plus_y_tf2), axis=-1)
preds



ctable.decode(preds[0], calc_argmax=False)

##Training a Dense network to add 2 numbers

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(4, input_dim=2, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])


In [ ]:
import numpy as np

# Generate random training data
X_train = np.random.randint(0, 100, size=(10000, 2))
y_train = X_train[:, 0] + X_train[:, 1]

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32)


In [ ]:
# Predict the sum of two numbers
X_test = np.array([[142, 118], [234, 99], [173, 27]])
y_pred = model.predict(X_test)

# Print the predictions
print(y_pred)